### Spark Session Creation

In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [65]:
spark = SparkSession.builder\
            .appName("MySparkApp")\
            .master("local[*]")\
            .getOrCreate()
print("Spark Version:", spark.version)


Spark Version: 4.0.1


### Data Sets

In [66]:
customers = [
    {"customer_id": 1, "name": "Alice", "age": 28, "city": "Bangalore"},
    {"customer_id": 2, "name": "Bob", "age": 34, "city": "Hyderabad"},
    {"customer_id": 3, "name": "Charlie", "age": 25, "city": "Mumbai"},
    {"customer_id": 4, "name": "Diana", "age": 30, "city": "Chennai"},
    {"customer_id": 5, "name": "Eve", "age": 27, "city": "Pune"}
]

orders = [
    {"order_id": 101, "customer_id": 1, "product": "Laptop", "amount": 75000, "order_date": "2025-09-10"},
    {"order_id": 102, "customer_id": 2, "product": "Smartphone", "amount": 25000, "order_date": "2025-09-12"},
    {"order_id": 103, "customer_id": 1, "product": "Keyboard", "amount": 2000, "order_date": "2025-09-15"},
    {"order_id": 104, "customer_id": 3, "product": "Monitor", "amount": 12000, "order_date": "2025-09-20"},
    {"order_id": 105, "customer_id": 5, "product": "Mouse", "amount": 1500, "order_date": "2025-09-25"},
    {"order_id": 106, "customer_id": 4, "product": "Headphones", "amount": 3000, "order_date": "2025-09-28"}
]

products = [
    {"product": "Laptop", "category": "Electronics", "in_stock": True},
    {"product": "Smartphone", "category": "Electronics", "in_stock": True},
    {"product": "Keyboard", "category": "Accessories", "in_stock": True},
    {"product": "Monitor", "category": "Electronics", "in_stock": False},
    {"product": "Mouse", "category": "Accessories", "in_stock": True},
    {"product": "Headphones", "category": "Accessories", "in_stock": True}
]


### Day-1


1️⃣ Problem: Create DataFrames

Objective: Create Spark DataFrames for all three datasets.

Difficulty: 🟢 Easy

Expected Output:

df_customers, df_orders, and df_products DataFrames displayed using show() and verified via printSchema().


In [67]:
customers_df=spark.createDataFrame(customers)
orders_df=spark.createDataFrame(orders)
products_df=spark.createDataFrame(products)

In [68]:
customers_df.show()

+---+---------+-----------+-------+
|age|     city|customer_id|   name|
+---+---------+-----------+-------+
| 28|Bangalore|          1|  Alice|
| 34|Hyderabad|          2|    Bob|
| 25|   Mumbai|          3|Charlie|
| 30|  Chennai|          4|  Diana|
| 27|     Pune|          5|    Eve|
+---+---------+-----------+-------+



In [69]:
orders_df.show()
orders_df.printSchema()


+------+-----------+----------+--------+----------+
|amount|customer_id|order_date|order_id|   product|
+------+-----------+----------+--------+----------+
| 75000|          1|2025-09-10|     101|    Laptop|
| 25000|          2|2025-09-12|     102|Smartphone|
|  2000|          1|2025-09-15|     103|  Keyboard|
| 12000|          3|2025-09-20|     104|   Monitor|
|  1500|          5|2025-09-25|     105|     Mouse|
|  3000|          4|2025-09-28|     106|Headphones|
+------+-----------+----------+--------+----------+

root
 |-- amount: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product: string (nullable = true)



In [70]:
products_df.show()

+-----------+--------+----------+
|   category|in_stock|   product|
+-----------+--------+----------+
|Electronics|    true|    Laptop|
|Electronics|    true|Smartphone|
|Accessories|    true|  Keyboard|
|Electronics|   false|   Monitor|
|Accessories|    true|     Mouse|
|Accessories|    true|Headphones|
+-----------+--------+----------+



2️⃣ Problem: Total Spend per Customer

Objective: Calculate the total amount spent by each customer.


In [71]:
group_df=orders_df.groupBy("customer_id").agg(sum("amount").alias("total_amount")).show()
group_df=orders_df.groupBy("customer_id").agg(sum("amount")).show()


+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|          1|       77000|
|          2|       25000|
|          3|       12000|
|          5|        1500|
|          4|        3000|
+-----------+------------+

+-----------+-----------+
|customer_id|sum(amount)|
+-----------+-----------+
|          1|      77000|
|          2|      25000|
|          3|      12000|
|          5|       1500|
|          4|       3000|
+-----------+-----------+



3️⃣ Problem: Join Customer & Order Details

Expected Output Columns: customer_id, name, city, product, amount, order_date

In [72]:
# df1.join(df2, on="column_name", how="join_type")
join_df=orders_df.join(customers_df,on='customer_id',how='left')
join_df.select('customer_id', 'name', 'city', 'product', 'amount', 'order_date').show()

+-----------+-------+---------+----------+------+----------+
|customer_id|   name|     city|   product|amount|order_date|
+-----------+-------+---------+----------+------+----------+
|          1|  Alice|Bangalore|    Laptop| 75000|2025-09-10|
|          2|    Bob|Hyderabad|Smartphone| 25000|2025-09-12|
|          1|  Alice|Bangalore|  Keyboard|  2000|2025-09-15|
|          3|Charlie|   Mumbai|   Monitor| 12000|2025-09-20|
|          5|    Eve|     Pune|     Mouse|  1500|2025-09-25|
|          4|  Diana|  Chennai|Headphones|  3000|2025-09-28|
+-----------+-------+---------+----------+------+----------+



4️⃣ Problem: Average Order Value by City

Objective: Find average order value (amount) per city.

In [73]:
group_join_df=join_df.groupBy('city').agg(avg('amount').alias('avg_amount'))
group_join_df.show()

+---------+----------+
|     city|avg_amount|
+---------+----------+
|Bangalore|   38500.0|
|  Chennai|    3000.0|
|   Mumbai|   12000.0|
|     Pune|    1500.0|
|Hyderabad|   25000.0|
+---------+----------+



5️⃣ Problem: Filter High-Value Orders

Objective: Filter orders with amount > 10,000 and sort descending by amount.

Expected Output Columns: order_id, customer_id, product, amount



In [74]:
join_df.filter('amount > 10000').select('order_id', 'customer_id', 'product', 'amount').show()

+--------+-----------+----------+------+
|order_id|customer_id|   product|amount|
+--------+-----------+----------+------+
|     101|          1|    Laptop| 75000|
|     102|          2|Smartphone| 25000|
|     104|          3|   Monitor| 12000|
+--------+-----------+----------+------+



6️⃣ Problem: Add Month Column

Objective: Extract month name from order_date and add as a new column.

Expected Output Columns: order_id, customer_id, order_date, order_month

In [75]:
join_df=join_df.withColumn('month',month('order_date'))
join_df.show()

+-----------+------+----------+--------+----------+---+---------+-------+-----+
|customer_id|amount|order_date|order_id|   product|age|     city|   name|month|
+-----------+------+----------+--------+----------+---+---------+-------+-----+
|          1| 75000|2025-09-10|     101|    Laptop| 28|Bangalore|  Alice|    9|
|          2| 25000|2025-09-12|     102|Smartphone| 34|Hyderabad|    Bob|    9|
|          1|  2000|2025-09-15|     103|  Keyboard| 28|Bangalore|  Alice|    9|
|          3| 12000|2025-09-20|     104|   Monitor| 25|   Mumbai|Charlie|    9|
|          5|  1500|2025-09-25|     105|     Mouse| 27|     Pune|    Eve|    9|
|          4|  3000|2025-09-28|     106|Headphones| 30|  Chennai|  Diana|    9|
+-----------+------+----------+--------+----------+---+---------+-------+-----+



7️⃣ Problem: Count of Products per Category

Objective: Find how many products belong to each category.

In [78]:
join_prod_df=join_df.join(products_df,on='product',how='left')
join_prod_df.show()

+----------+-----------+------+----------+--------+---+---------+-------+-----+-----------+--------+
|   product|customer_id|amount|order_date|order_id|age|     city|   name|month|   category|in_stock|
+----------+-----------+------+----------+--------+---+---------+-------+-----+-----------+--------+
|    Laptop|          1| 75000|2025-09-10|     101| 28|Bangalore|  Alice|    9|Electronics|    true|
|Smartphone|          2| 25000|2025-09-12|     102| 34|Hyderabad|    Bob|    9|Electronics|    true|
|  Keyboard|          1|  2000|2025-09-15|     103| 28|Bangalore|  Alice|    9|Accessories|    true|
|   Monitor|          3| 12000|2025-09-20|     104| 25|   Mumbai|Charlie|    9|Electronics|   false|
|     Mouse|          5|  1500|2025-09-25|     105| 27|     Pune|    Eve|    9|Accessories|    true|
|Headphones|          4|  3000|2025-09-28|     106| 30|  Chennai|  Diana|    9|Accessories|    true|
+----------+-----------+------+----------+--------+---+---------+-------+-----+-----------+

In [83]:
join_prod_df.groupBy('category').agg(count('category').alias('count_items')).show()

+-----------+-----------+
|   category|count_items|
+-----------+-----------+
|Electronics|          3|
|Accessories|          3|
+-----------+-----------+



8️⃣ Problem: Orders with Product Category

Objective: Join orders with products on product and show full details.

Columns: order_id, customer_id, product, category, amount, in_stock

In [85]:
join_prod_df[['order_id', 'customer_id', 'product', 'category', 'amount', 'in_stock']].show()


+--------+-----------+----------+-----------+------+--------+
|order_id|customer_id|   product|   category|amount|in_stock|
+--------+-----------+----------+-----------+------+--------+
|     101|          1|    Laptop|Electronics| 75000|    true|
|     102|          2|Smartphone|Electronics| 25000|    true|
|     103|          1|  Keyboard|Accessories|  2000|    true|
|     104|          3|   Monitor|Electronics| 12000|   false|
|     105|          5|     Mouse|Accessories|  1500|    true|
|     106|          4|Headphones|Accessories|  3000|    true|
+--------+-----------+----------+-----------+------+--------+



9️⃣ Problem: Customers Without Orders

Objective: List customers who didn’t place any orders.

Expected Output: (empty in this dataset, but still good practice)

In [91]:
join_prod_df[~(join_prod_df['order_id']>102)].show()


+----------+-----------+------+----------+--------+---+---------+-----+-----+-----------+--------+
|   product|customer_id|amount|order_date|order_id|age|     city| name|month|   category|in_stock|
+----------+-----------+------+----------+--------+---+---------+-----+-----+-----------+--------+
|    Laptop|          1| 75000|2025-09-10|     101| 28|Bangalore|Alice|    9|Electronics|    true|
|Smartphone|          2| 25000|2025-09-12|     102| 34|Hyderabad|  Bob|    9|Electronics|    true|
+----------+-----------+------+----------+--------+---+---------+-----+-----+-----------+--------+



🔟 Problem: Total Spend per Category

Objective: Calculate total order amount per product category (Accessories/Electronics).

Expected Output Example:

In [92]:
join_prod_df.groupBy('category').agg(sum('amount').alias('total_order_amount')).show()

+-----------+------------------+
|   category|total_order_amount|
+-----------+------------------+
|Electronics|            112000|
|Accessories|              6500|
+-----------+------------------+

